# Reading datasets forming a new one

In [5]:
import pandas as pd
import os

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

directory = 'C:/Users/99451/Desktop/MODEL/eng_prem'

dfs = []

# Define the columns you want to extract from each CSV file
columns_to_keep = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'B365H', 'B365D', 'B365A']

for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        df = pd.read_csv(os.path.join(directory, filename), on_bad_lines = 'skip', encoding='latin-1')
        df = df[columns_to_keep]
        dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

print("Number of rows:", df.shape[0])
print("Number of columns:", df.shape[1])
df.head()

Number of rows: 8214
Number of columns: 9


,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A
0,17/08/2002,Blackburn,Sunderland,0.0,0.0,D,1.727,3.25,4.333
1,17/08/2002,Charlton,Chelsea,2.0,3.0,A,2.800,3.25,2.200
2,17/08/2002,Everton,Tottenham,2.0,2.0,D,2.250,3.25,2.750
3,17/08/2002,Fulham,Bolton,4.0,1.0,H,1.727,3.25,4.333
4,17/08/2002,Leeds,Man City,3.0,0.0,H,1.667,3.40,4.500


# Adding Moving Averages

In [7]:
df['FTHGS'] = df.groupby('HomeTeam')['FTHG'].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))
df['FTHGC'] = df.groupby('HomeTeam')['FTAG'].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))

df['FTAGS'] = df.groupby('AwayTeam')['FTAG'].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))
df['FTAGC'] = df.groupby('AwayTeam')['FTHG'].transform(lambda x: x.rolling(5, min_periods=1).mean().shift(1))

total = 1 / df['B365H'] + 1 / df['B365D'] + 1 / df['B365A']

df['FTH'] = (1 / df['B365H']) / total
df['FTD'] = (1 / df['B365D']) / total
df['FTA'] = (1 / df['B365A']) / total
df['FTDA'] = df['FTD'] + df['FTA']

# Calculate the league-wide moving average for home_goals and away_goals
df['league_home_goals'] = df['FTHG'].rolling(50, min_periods=1).mean().shift(1)
df['league_away_goals'] = df['FTAG'].rolling(50, min_periods=1).mean().shift(1)

df.tail()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,B365H,B365D,B365A,FTHGS,FTHGC,FTAGS,FTAGC,FTH,FTD,FTA,FTDA,league_home_goals,league_away_goals
8209,13/04/2024,Bournemouth,Man United,2.0,2.0,D,2.40,3.90,2.6,1.8,1.4,1.8,2.0,0.393939,0.242424,0.363636,0.606061,1.92,1.30
8210,14/04/2024,Liverpool,Crystal Palace,0.0,1.0,A,1.22,7.00,12.0,2.6,1.0,0.8,2.0,0.783728,0.136593,0.079679,0.216272,1.92,1.34
8211,14/04/2024,West Ham,Fulham,0.0,2.0,A,2.38,3.60,2.8,1.6,2.4,1.8,2.2,0.398230,0.263274,0.338496,0.601770,1.88,1.32
8212,14/04/2024,Arsenal,Aston Villa,0.0,2.0,A,1.29,5.75,10.0,3.2,0.6,2.4,1.6,0.738908,0.165772,0.095319,0.261092,1.86,1.34
8213,15/04/2024,Chelsea,Everton,6.0,0.0,H,1.70,4.20,4.5,2.4,2.2,0.6,1.6,0.560997,0.227070,0.211932,0.439003,1.82,1.36
